<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [1]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

C:\Users\Alberto\Anaconda3\envs\clase\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Alberto\Anaconda3\envs\clase\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Alberto\Anaconda3\envs\clase\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# datos
data=pd.read_csv('data\cookie_cats.csv')
data.head()


,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [3]:
# transformacion
len(data.userid.unique())-data.shape[0] # No repeated users


0

In [4]:
data.isnull().sum() # No null values

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [5]:
# muestreo





In [6]:
# testeo para retention_1
gate_30_true=data[data.version=='gate_30']['retention_1'].value_counts()[1]
gate_30_false=data[data.version=='gate_30']['retention_1'].value_counts()[0]
gate_40_true=data[data.version=='gate_40']['retention_1'].value_counts()[1]
gate_40_false=data[data.version=='gate_30']['retention_1'].value_counts()[0]
# Frecuency

In [7]:
gate_30_mean=gate_30_true/len(data[data.version=='gate_30']['retention_1'])
gate_30_mean # relative frequency of retention 1 event in gate_30 version

0.4481879194630872

In [8]:
gate_40_mean=gate_40_true/len(data[data.version=='gate_40']['retention_1'])
gate_40_mean # relative frequency of retention 1 event in gate_40 version

0.44228274967574577

In [9]:
from calc_prob import calc_prob_between
from scipy.stats import beta


In [16]:
beta_T=beta(gate_40_true+1,gate_40_false+1)
beta_C=beta(gate_30_true+1,gate_30_false+1)

In [17]:
prob=calc_prob_between(beta_T, beta_C)


In [18]:
# conclusiones
# Explica detalladamente las conclusiones obtenidas de. tu testeo.

In [19]:
print (f"Test option lift Conversion Rates by {gate_40_mean-gate_30_mean:2.2f}% with {prob*100:2.1f}% probability.")


Test option lift Conversion Rates by -0.01% with 62.4% probability.
